<a href="https://colab.research.google.com/github/Bellajia721/WeatherWise_A2/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [1]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install requests
!pip install matplotlib
!pip install pyinputplus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existin

In [55]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: d6ee01b95e1b492baa06b8d295907578.EN_MdtgVL5KhHIQtW024XQJJ


## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import json
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response
# Add any other setup code here
from ipywidgets import Text, DatePicker, Button, VBox
from IPython.display import display

from hands_on_ai.chat import get_response

def ai_extend_forecast(weather_data: dict, extra_days: int = 2):
    if not weather_data or "daily" not in weather_data or len(weather_data["daily"]) == 0:
        print("⚠️ No base daily data to extend.")
        return weather_data

    base_days = weather_data["daily"][-3:]
    compact = [
        {
            "date": d.get("date"),
            "max_temp": d.get("maxtempC"),
            "min_temp": d.get("mintempC"),
            "average_temp": d.get("avgtempC"),
            "total_precip": d.get("totalprecipMM"),
        }
        for d in base_days
    ]

    prompt = f"""
    You are a cautious weather assistant.
    Based on the last real 3 daily records below:
    {compact}
    Estimate the next {extra_days} days' weather with realistic smooth trends.
    Return ONLY JSON list with keys: date, max_temp, min_temp, average_temp, total_precip.
    """

    raw = get_response(prompt)
    print("AI raw output:", raw)

    import json
    try:
        estimates = json.loads(raw)
    except:
        print("⚠️ Model output not JSON — skipping extension.")
        return weather_data


    from datetime import datetime, timedelta
    last_date = datetime.strptime(base_days[-1]["date"], "%Y-%m-%d").date()
    for i, e in enumerate(estimates):
        e["date"] = (last_date + timedelta(days=i + 1)).isoformat()
        e["ai_estimated"] = True
        weather_data["daily"].append(e)

    return weather_data

city_input = Text(description = 'City:', placeholder = 'e.g. Perth')
date_input = DatePicker(description = 'Date:')
submit_button = Button(description='Get Weather', button_style='success')

def on_submit_button_click(b):
    city = city_input.value
    date = date_input.value.strftime('%m-%d-%y')
    print(f'City: {city}')
    print(f'Date: {date}')

submit_button.on_click(on_submit_button_click)

display(VBox([city_input, date_input, submit_button]))

## 🌤️ Weather Data Functions

In [57]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    try:
      raw = get_weather(location = location, format="raw_json", use_mock = False)
      if isinstance(raw, str) and raw.startswith("Error:"):
        print(f"Weather API returned an error response: {raw}")
        return None
    except Exception as e:
      print(f"An error occurred: {e}")
      return None

    current_weather = (raw.get("current_condition") or [{}])[0]
    desc = None
    if current_weather.get("weatherDesc"):
      desc = (current_weather["weatherDesc"][0] or {}).get("value")
    forecast_weather = (raw.get("weather") or [])[:forecast_days]

    return {
        "location": location,
        "current_weather": {
            "temperature": current_weather.get("temp_C"),
            "windspeed": current_weather.get("windspeedKmph"),
            "humidity": current_weather.get("humidity"),
            "precipitation": current_weather.get("precipMM"),
            "descripption": desc
        },
        "daily":[
            {
                 "date": d.get("date"),
                 "average_temp": d.get("avgtempC"),
                 "windspeed": d.get("MaxwindKmph"),
                 "humidity": d.get("avghumidity"),
                 "total_precip": d.get("totalprecipMM"),
                 "max_temp": d.get("maxtempC"),
                 "min_temp": d.get("mintempC"),
        }
        for d in forecast_weather
      ]
    }

## 📊 Visualisation Functions

In [58]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if not "daily" in weather_data:
      print("No temperature data available")
      return None

    days = weather_data["daily"]
    dates = [d["date"] for d in days]
    avg_temp = [float(d["average_temp"]) for d in days]
    max_temp = [float(d["max_temp"]) for d in days]
    min_temp = [float(d["min_temp"]) for d in days]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(dates, avg_temp, label ='Average Temperature(°C)', color = 'green', marker ='o')
    ax.plot(dates, max_temp, label ='Maximum Temperature(°C)', color = 'red', marker ='^')
    ax.plot(dates, min_temp, label ='Minimum Temperature(°C)', color = 'blue', marker ='v')
    ax.set_xlabel('Date')
    ax.set_ylabel('Temperature (°C)')
    ax.set_title('Temperature Trends')
    ax.legend()
    ax.grid(True)
    plt.xticks(rotation=30)

    if output_type == 'figure':
      return fig
    else:
      plt.show()


In [59]:
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if not "daily" in weather_data:
      print("No precipitation data available")
      return None
    days = weather_data["daily"]
    dates = [d["date"] for d in days]
    total_precip = [float(d.get("total_precip") or 0) for d in days]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(dates, total_precip, label='Total Precipitation (mm)', color='darkblue', marker='s')
    ax.set_xlabel('Date')
    ax.set_ylabel('Precipitation (mm)')
    ax.set_title('Precipitation Trends')
    ax.legend()
    ax.grid(True)
    plt.xticks(rotation=30)

    if output_type == 'figure':
      return fig
    else:
      plt.show()

In [63]:
def creat_windspeed_visualisation(weather_data, output_type='display'):
  if not "daily" in weather_data:
    print("No windspeed data available")
    return None
  days = weather_data["daily"]
  dates = [d["date"] for d in days]
  windspeed = [float(d.get("windspeed") or 0) for d in days]

  fig, ax = plt.subplots(figsize=(10, 6))
  ax.plot(dates, windspeed, label='Wind Speed (km/h)', color='purple', marker='D')
  ax.set_xlabel('Date')
  ax.set_ylabel('Wind Speed (km/h)')
  ax.set_title('Wind Speed Trends')
  ax.legend()
  ax.grid(True)
  plt.xticks(rotation=30)

  if output_type == 'figure':
    return fig
  else:
    plt.show()

In [64]:
def creat_humidity_visualisation(weather_data, output_type='display'):
  if not "daily" in weather_data:
    print("No humidity data available")
    return None
  days = weather_data["daily"]
  dates = [d["date"] for d in days]
  humidity = [float(d.get("humidity") or 0) for d in days]
  fig, ax = plt.subplots(figsize=(10, 6))
  ax.plot(dates, humidity, label='Humidity (%)', color='orange', marker='h')
  ax.set_xlabel('Date')
  ax.set_ylabel('Humidity (%)')
  ax.set_title('Humidity Trends')
  ax.legend()
  ax.grid(True)
  plt.xticks(rotation=30)

  if output_type == 'figure':
    return fig
  else:
    plt.show()

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.